In [36]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import pandas as pd
import numpy as np
import os
from PIL import Image
import shutil
from sklearn.metrics import accuracy_score

In [6]:
def convert_to_jpeg(file_path):
    try:
        img = Image.open(file_path)
        rgb_img = img.convert('RGB')
        rgb_img.save(file_path, format='JPEG', quality=90)
    except Exception as e:
        print(f"Error converting file {file_path} to JPEG: {e}")


In [7]:

main_dir = '/kaggle/input/products/train/train'
writable_dir = '/kaggle/working/train'


shutil.copytree(main_dir, writable_dir)


for subdir, dirs, files in os.walk(writable_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        convert_to_jpeg(file_path)

In [8]:

main_dir = '/kaggle/input/products/validation/validation'
writable_dir = '/kaggle/working/validation'


shutil.copytree(main_dir, writable_dir)


for subdir, dirs, files in os.walk(writable_dir):
    for file in files:
        file_path = os.path.join(subdir, file)
        convert_to_jpeg(file_path)

In [5]:
train_dir = '/kaggle/working/train'
val_dir = '/kaggle/working/validation'

In [9]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.1, 0.9]
)


In [10]:
kernel_size = (3,3)
pool_size = (2,2)
first_filters = 32
second_filters = 64
third_filtesr = 128


In [11]:
dropout_conv = 0.3
dropout_dense = 0.3

In [12]:
IMAGE_SIZE = 240

CLASSES = 4

In [13]:
#Model setup
model = Sequential()
model.add(Conv2D(first_filters,kernel_size,activation='relu',input_shape = (IMAGE_SIZE,IMAGE_SIZE,3))) #This 3, would be a 1 if the images are gray
model.add(Conv2D(first_filters,kernel_size,activation='relu'))
model.add(Conv2D(first_filters,kernel_size,activation='relu'))
model.add(MaxPool2D(pool_size=pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters,kernel_size,activation='relu'))
model.add(Conv2D(second_filters,kernel_size,activation='relu'))
model.add(Conv2D(second_filters,kernel_size,activation='relu'))
model.add(MaxPool2D(pool_size=pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filtesr,kernel_size,activation='relu'))
model.add(Conv2D(third_filtesr,kernel_size,activation='relu'))
model.add(Conv2D(third_filtesr,kernel_size,activation='relu'))
model.add(MaxPool2D(pool_size=pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(dropout_dense))
model.add(Dense(4, activation='softmax'))



/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [15]:
model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 238, 238, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 236, 236, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 234, 234, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 117, 117, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 117, 117, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 115, 115, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 113, 113, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 111, 111, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 55, 55, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 53, 53, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 51, 51, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 49, 49, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 24, 24, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 73728)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    18,874,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,356,420 (73.84 MB)

 Trainable params: 19,356,420 (73.84 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
batch_size = 16

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [19]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [24]:
filepath = '/kaggle/working/eshta.keras'

In [25]:
checkpoint = ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1, save_best_only=True,mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=0.5, patience=3, verbose=1, mode='max',min_lr=0.00001)
callsback = [checkpoint, reduce_lr]

In [26]:
train_generator = train_datagen.flow_from_directory(
    '/kaggle/working/train', 
    target_size=(IMAGE_SIZE, IMAGE_SIZE), 
    batch_size=batch_size,
    class_mode='categorical'  
)


validation_generator = test_datagen.flow_from_directory(
    '/kaggle/working/validation',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 428 images belonging to 4 classes.
Found 90 images belonging to 4 classes.


In [31]:
class_names = list(train_generator.class_indices.keys())

In [28]:
model.fit(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=800 // batch_size,
    callbacks=callsback
)

Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/125 ━━━━━━━━━━━━━━━━━━━━ 1:15:44 37s/step - accuracy: 0.3125 - loss: 1.3807

I0000 00:00:1710478242.924107     184 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710478242.945765     184 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


  2/125 ━━━━━━━━━━━━━━━━━━━━ 51:32 25s/step - accuracy: 0.3348 - loss: 1.3775  

W0000 00:00:1710478268.119872     187 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 27/125 ━━━━━━━━━━━━━━━━━━━━ 1:53 1s/step - accuracy: 0.2872 - loss: 1.3905

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
W0000 00:00:1710478273.996325     185 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Epoch 1: val_accuracy improved from -inf to 0.43333, saving model to /kaggle/working/eshta.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 75s 313ms/step - accuracy: 0.2782 - loss: 1.3885 - val_accuracy: 0.4333 - val_loss: 1.3759 - learning_rate: 1.0000e-04
Epoch 2/50
 27/125 ━━━━━━━━━━━━━━━━━━━━ 18s 187ms/step - accuracy: 0.3052 - loss: 1.3720
Epoch 2: val_accuracy improved from 0.43333 to 0.46667, saving model to /kaggle/working/eshta.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 52ms/step - accuracy: 0.3352 - loss: 1.3632 - val_accuracy: 0.4667 - val_loss: 1.3103 - learning_rate: 1.0000e-04
Epoch 3/50
 27/125 ━━━━━━━━━━━━━━━━━━━━ 18s 187ms/step - accuracy: 0.3371 - loss: 1.3071
Epoch 3: val_accuracy improved from 0.46667 to 0.61111, saving model to /kaggle/working/eshta.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - accuracy: 0.3842 - loss: 1.2666 - val_accuracy: 0.6111 - val_loss: 1.0403 - learning_rate: 1.0000e-04
Epoch 4/50
 27/125 ━━━━━━━━━━━━━━━━━━━━ 19s 195ms/step - accuracy: 0.5178 - loss: 1.

In [39]:

test_folder_path = '/kaggle/working/validation'

y_true = []
y_pred = []

for class_folder in os.listdir(test_folder_path):
    class_folder_path = os.path.join(test_folder_path, class_folder)
    
    
    for img_file in os.listdir(class_folder_path):
        img_path = os.path.join(class_folder_path, img_file)
        new_image = load_image(img_path, size=(IMAGE_SIZE, IMAGE_SIZE))
        
        
        pred = model.predict(new_image)
        predicted_class = class_names[pred.argmax()]
        y_true.append(class_folder)
        y_pred.append(predicted_class)


accuracy = accuracy_score(y_true, y_pred)

print(f'Test accuracy: {accuracy * 100:.2f}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━